# BassaredHoyerMoscaTrapp Algorithm

### Introduction
Grover's Algorithm requires the knowledge of the number of marked elements (the number of elements *x* such that *f(x) = 1*), so that the Diffusion Operator could be ran the optimal number of times.

What happens if the number of marked elements is not known? 
**It is still possible to achieve a quadratic speedup.**

In this notebook, the function *QSearch()* is implemented. At no line in the function *QSearch()* does the program know the number of marked elements. In the implementation, only in *GroverOracle()* (an oracle which *QSearch()* calls) does the program know the number of marked elements.

In the end, what should be observed is that the number of calls to the *GroverOracle()* is in the magnitude of $\sqrt{N}$.

In [148]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from math import pi
import math
import random
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from qiskit import IBMQ, Aer, BasicAer, execute, QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info import state_fidelity
from qiskit.tools.visualization import circuit_drawer
from qiskit.circuit.library.generalized_gates.mcmt import MCMT
from qiskit.circuit.library.standard_gates.x import XGate, HGate
from qiskit.circuit.library import ZGate
from qiskit.visualization import plot_histogram

### Defining Variables and Functions
Firstly, we define:
1. *n* : the number of qubits
2. *good_set* : the set of all marked elements
3. *f(x)* : returns *True* if *x* is in *good_set*, *False* otherwise
4. *convert_to_int(s)* : converts a string *s* to integer, treating *s[0]* as the least significant bit

In [146]:
# Using n = 10 means that at the end, the number of calls to oracle should be 
# in the order of ~33

n = 10
good_set = [2, 22, 42]
simulator = Aer.get_backend('qasm_simulator')

In [147]:
def f(x):
    return (x in good_set)


def convert_to_int(s):
    ans = 0
    for i in range(len(s)):
        ans = ans + (2**i)*int(s[len(s) - i - 1])
    return ans

### Using Some Oracles
The following oracles (*AndOracle*, *ControlledOnIntOracle*, *DiffusionOperator*, *GroverOracle*) are taken from *Oracles.py*. Please refer to *Oracles.py* if you find these oracles unfamiliar.

In [125]:
def AndOracle(given_circuit, nn, target_qubit):
    given_circuit.barrier()
    controlled_xgate = XGate().control(nn)
    curlist = list(range(0, nn))
    curlist.append(target_qubit)
    given_circuit.append(controlled_xgate, curlist)
    given_circuit.barrier()


def ControlledOnIntOracle(given_circuit, nn, target_qubit, x, reverse=False):
    
    if (x != 0 and nn < int(math.floor(math.log2(x) + 1))):
        print("Not enough qubits to do ControlledOnIntOracle\n")
        return
    
    given_circuit.barrier()
    xx = x
    counter = 0
    for i in range(nn):
        if (xx%2 == 0):
            given_circuit.x(counter)
        xx = (xx - (xx%2))/2
        counter = counter + 1
    AndOracle(given_circuit, nn, target_qubit)
    
    xx = x
    counter = 0
    for i in range(nn):
        if (xx%2 == 0):
            given_circuit.x(counter)
        xx = (xx - (xx%2))/2
        counter = counter + 1
    given_circuit.barrier()

    
def DiffusionOperator(given_circuit, nn, name = "DiffusionOperator"):
    
    given_circuit.h(range(nn))
    given_circuit.x(range(nn))
    controlled_gate = ZGate().control(nn - 1)
    given_circuit.append(controlled_gate, list(range(0, nn)))
    given_circuit.x(range(nn))
    given_circuit.h(range(nn))

    
def GroverOracle(given_circuit, nn, name = 'GroverOracle'):
    for i in good_set:
        ControlledOnIntOracle(given_circuit, nn, nn, i)

### QSearch PseudoCode
1. Set *l = 0* and let *c* be any constant such that *1 < c < 2*.
2. Set *l = l + 1* and set *M = $\lceil c^l \rceil$*.
3. Apply *H* on the initial state *|0>*.
4. Pick random integer *j* between *1* and *M*.
5. Apply *GroverOracle()* and *DiffusionOperator()* *j* times to the register.
6. Measure. If the outcome *|z>* is marked, then output *z*. Otherwise, repeat from Step 2.

In [155]:
# nn = number of qubits
# c = random constant between 1 and 2 exclusive

def QSearch(nn, c=1.5):
    l = 0
    print("The value of c is: " + str(c) + ".")
    
    done = False
    total = 0
    while (done == False):
        l = l + 1
        m = math.ceil(c**l)
        print("l = " + str(l) + ": ")
        print("m = " + str(m) + ".")
        
        qc = QuantumCircuit(nn + 1, nn)
        qc.h(range(nn))
        j = random.randint(1, m)
        
        for i in range(j):
            total = total + 1
            GroverOracle(qc, nn)
            DiffusionOperator(qc, nn)
        
        qc.measure(range(nn), range(nn))
        
        #Gets a single measurement
        counts = execute(qc, simulator, shots = 1).result().get_counts(qc)
        
        y = convert_to_int(list(counts.keys())[0])
        print("Measured and currently trying y = " + str(y) + ".")
        
        if (f(y) == True):
            print("SUCCESS")
            done = True
            return (y, counts, total)
        
        del qc

In [156]:
(value, cc, total_counts) = QSearch(n, 1 + random.random())
print("\n")
print("One correct output is: " + str(value) + ".")
print("QSearch called oracle a total of " + str(total_counts) + " times.")

The value of c is: 1.9242719874382423.
l = 1: 
m = 2.
Measured and currently trying y = 796.
l = 2: 
m = 4.
Measured and currently trying y = 636.
l = 3: 
m = 8.
Measured and currently trying y = 772.
l = 4: 
m = 14.
Measured and currently trying y = 381.
l = 5: 
m = 27.
Measured and currently trying y = 2.
SUCCESS


One correct output is: 2.
QSearch called oracle a total of 31 times.


### Final Note
A key point to note is that, inside the function *QSearch()*, there is no place to get information on how many marked elements there are. The program can only make calls to *GroverOracle()*, which has access to *good_set*.

The number of calls made to the oracle is in the order of $\sqrt{2^n}$.

### References
1. Brassard, Hoyer, Mosca, Trapp (2000): https://arxiv.org/abs/quant-ph/0005055